In [ ]:
import pandas as pd
import numpy as np
import  matplotlib.pyplot as plt
import seaborn as sns
from scipy import interpolate

from scipy.interpolate import interp1d
from numpy.polynomial.polynomial import Polynomial
from sklearn.ensemble import RandomForestRegressor

In [ ]:
trainFeatures = pd.read_csv('data/train_features.csv')
testFeatures = pd.read_csv('data/test_features.csv')
trainLabels = pd.read_csv('data/train_labels.csv')
example = pd.read_csv('data/submission_format.csv')

In [ ]:
trainFeatures.head()

In [ ]:
trainFeatures.shape

In [ ]:
testFeatures.head()

In [ ]:
example.head()

# Data Assessing

## Informasi Umum Dataset Train Features

In [ ]:
trainFeatures.info()

In [ ]:
trainFeatures.describe()

## Cek Nilai Null Dataset Train Features

In [ ]:
trainFeatures.isnull().sum()

## Cek Nilai Duplikat Dataset Train Features

In [ ]:
trainFeatures.duplicated().sum()

# Data Cleaning

## Kolom Pendidikan

In [ ]:
# Cek Nilai Unique
trainFeatures['pendidikan'].unique()

Terdapat dua nilai salah yaitu nan dan '5'. Maka perlu dibersihkan pada dua nilai tersebut.

### Olah Data dengan Nilai Unique '5'

In [ ]:
# Ambil contoh nilai pada kolom pendidikan dengan nilai '5'
trainFeatures[trainFeatures['pendidikan'] == '5'].head()

In [ ]:
# Mengambil indeks data dengan nilai unique '5'
dropPendidikan = trainFeatures[trainFeatures['pendidikan'] == '5'].index
# Menghapus nilai dengan indeks tersebut
trainFeatures.drop(dropPendidikan, inplace=True)

Saat ini nilai dengan inputan '5' sudah terhapus, karena hanya terdapat 2 data saja maka dapat dikatakan aman untuk dihapus.

Selanjutnya yaitu menangani data dengan inputan nan. Untuk langkah yang diambil yaitu mengecek seberapa banyak nilai nan yang ada. Jika jumlah terbilang sedikit, langkah yang diambil adalah menghapus nilai tersebut (seperti pada inputan '5' sebelumnya). Namun, jika jumlahnya terbilang cukup banyak, maka langkah yang diambil yaitu mengubah setiap nilai nan menjadi nilai modus pada kolom **pendidikan**.

### Olah Data dengan Nilai Unique NaN

In [ ]:
# Mengambil sample data dengan inputan nan
dataNanPendidikan = trainFeatures[trainFeatures['pendidikan'].isnull()]

# Mengecek Banyak Baris Data
print('Banyak Data dengan inputan nan:', dataNanPendidikan.shape[0], '\n')
print('Dengan sample data sebagai berikut: ')
dataNanPendidikan.sample(10)

Dikarenakan jumlah dari data nan cukup banyak, maka akan dilakukan pengubahan isi dengan data modus pada kolom **pendidikan**.

In [ ]:
# Mengambil jumlah isi terbanyak pada kolom pendidikan
modePendidikan = trainFeatures['pendidikan'].mode()[0]
print('Nilai dengan modus terbanyak adalah', modePendidikan)

In [ ]:
# Mengubah nilai nan menjadi nilai modus
trainFeatures['pendidikan'] = trainFeatures['pendidikan'].fillna(modePendidikan)

In [ ]:
# Cek ulang nilai unique
trainFeatures['pendidikan'].unique()

Pada saat ini, kondisi kolom **pendidikan** sudah normal.

## Kolom Status Pernikahan

In [ ]:
# Cek Nilai Unique
trainFeatures['status_pernikahan'].unique()

Mirip dengan kolom Pendidikan, terdapat dua nilai tidak sesuai pada kolom **Status Pernikahan** yaitu '5' dan nan. Maka perlu dilakukan proses yang sama dengan kolom Pendidikan yaitu meninjau setiap invalid value yang ada untuk dilakukan langkah lebih lanjut.

### Olah Data dengan Nilai Unique '5'

In [ ]:
# Mengambil sample data dengan nilai unique '5'
trainFeatures[trainFeatures['status_pernikahan'] == '5']

Dikarenakan hanya terdapat satu nilai saja dengan value '5', maka dapat dihapus saja karena tidak terlalu berpengaruh terhadap keseluruhan data.

In [ ]:
# Mengambil data dengan nilai unique '5'
dropStatusNikah = trainFeatures[trainFeatures['status_pernikahan'] == '5'].index
# Menghapus data
trainFeatures.drop(dropStatusNikah, inplace=True)


### Olah Data dengan Nilai Unique NaN

In [ ]:
# Mengambil sample data dengan inputan nan
dataNanPernikahan = trainFeatures[trainFeatures['status_pernikahan'].isnull()]

# Mengecek Banyak Baris Data
print('Banyak Data dengan inputan nan:', dataNanPernikahan.shape[0], '\n')
print('Dengan sample data sebagai berikut: ')
dataNanPernikahan.sample(10)

Dikarenakan jumlah dari data nan cukup banyak, maka akan dilakukan pengubahan isi data nan dengan analisis lebih lanjut pada kolom-kolom yang berpotensi memiliki korelasi dengan kolom **status pernikahan**. 

Yaitu dengan melihat kolom **jumlah anak balita** dan **jumlah anak remaja** dengan asumsi jika memiliki nilai lebih dari 0 dari kedua kolom tersebut, maka akan diubah menjadi "Menikah".

In [ ]:
# Ubah NaN menjadi String Terlebih Dahulu
trainFeatures['status_pernikahan'].fillna('kosong', inplace=True)

# Buat Fungsi
def ubahPernikahan(row):
    if row['status_pernikahan'] == 'kosong':
        if row['jumlah_anak_balita'] > 0 or row['jumlah_anak_remaja'] > 0:
            return 'Menikah'
        else:
            return 'Sendiri'
    else:
        return row['status_pernikahan']

trainFeatures['status_pernikahan'] = trainFeatures.apply(ubahPernikahan, axis=1)

In [ ]:
trainFeatures['status_pernikahan'].unique()

Saat ini nilai unique pada kolom **status pernikahan** sudah normal dan tidak ada nilai nan.

## Kolom Pendapatan

In [ ]:
trainFeatures.pendapatan.sample(5)

### Cek Nilai Statistik 

In [ ]:
trainFeatures.describe()

### Pengecekan Nilai Null

Dari hasil analisis sebelumnya, terlihat bahwa terdapat nilai null pada kolom ini.

In [ ]:
nullPendapatan = trainFeatures[trainFeatures['pendapatan'].isnull()]
nullPendapatan.head(10)

In [ ]:
trainFeatures.sample(10)

Yang dilakukan adalah menggunakan interpolasi polinomial untuk menutup setiap nilai NaN yang ada dengan titik terdekat data ke data lainnya secara linear.

In [ ]:
# Data x dan y yang tidak null
x_known = trainFeatures.index[~trainFeatures['pendapatan'].isnull()]
y_known = trainFeatures.loc[x_known, 'pendapatan']

# Buat objek interpolasi polinomial orde rendah (misalnya, orde 2)
poly_interp = Polynomial.fit(x_known, y_known, deg=3)

# Memasang indeks titik data yang akan diisi (NaN)
fill_indices = trainFeatures.index[trainFeatures['pendapatan'].isnull()]

# Isi nilai-nilai NaN dengan hasil interpolasi polinomial
trainFeatures.loc[fill_indices, 'pendapatan'] = poly_interp(fill_indices)


Pada saat ini sudah tidak terdapat nilai NaN lagi pada kolom **pendapatan**, sehingga dapat dilanjutkan pada proses analisis selanjutnya.

In [ ]:
trainFeatures.pendapatan.describe()

## Kolom Jumlah Anak Balita

### Cek Nilai Unique

In [ ]:
trainFeatures['jumlah_anak_balita'].unique()

Pada pengecekan nilai unique di atas, ditemukan nilai NaN yang perlu diganti dengan nilai lain.

### Hapus Nilai Null

In [ ]:
trainFeatures['jumlah_anak_balita'].fillna(0, inplace=True)
trainFeatures['jumlah_anak_balita'] = trainFeatures['jumlah_anak_balita'].astype(int)
print(trainFeatures['jumlah_anak_balita'].dtype)

In [ ]:
# Cek Nilai Unique Ulang
trainFeatures['jumlah_anak_balita'].unique()

Pada proses ini, langkah yang diambil adalah mengubah nilai NaN menjadi **0** dengan asumsi setiap nilai NaN diartikan baris data penduduk tersebut memiliki jumlah anak balita sebanyak **0**.

## Kolom Jumlah Anak Remaja

### Cek Nilai Unique

In [ ]:
trainFeatures['jumlah_anak_remaja'].unique()

Sama seperti pada kasus kolom *jumlah anak balita*, terdapat nilai NaN pada kolom *jumlah anak remaja* yang perlu diubah dengan nilai lainnya.

### Hapus Nilai Null

In [ ]:
trainFeatures['jumlah_anak_remaja'].fillna(0, inplace=True)
trainFeatures['jumlah_anak_remaja'] = trainFeatures['jumlah_anak_remaja'].astype(int)
print(trainFeatures['jumlah_anak_remaja'].dtype)

In [ ]:
# Cek Nilai Unique Ulang
trainFeatures['jumlah_anak_remaja'].unique()

Hasil pada kolom *jumlah anak remaja* juga mirip dengan hasil kolom *jumlah anak balita*, yaitu dengan mengubah nilai NaN dengan nilai **0**. Sebagai asumsi bahwa nilai NaN berarti baris data penduduk tersebut tidak memiliki anak remaja.

## Kolom Terakhir Belanja

### Cek Nilai Unique

In [ ]:
trainFeatures['terakhir_belanja'].unique()

Pada pengecekan nilai unique, hasilnya terdapat nilai NaN yang terkandung didalam kolom *terakhir belanja*. Maka perlu dilakukan pengecekan batas minimum dan maksimum data serta nilai statistik lainnya seperti rata-rata untuk mengubah nilai NaN ini dengan sajian nilai Unique yang lebih rapi.

### Cek Nilai Min dan Max

In [ ]:
nilaiMax = trainFeatures.terakhir_belanja.max()
nilaiMin = trainFeatures.terakhir_belanja.min()

print('Nilai maksimal dari kolom terakhir belanja adalah', nilaiMax)
print('Nilai minimal dari kolom terakhir belanja adalah', nilaiMin)

Dikarenakan tidak adanya acuan data tambahan dari kolom *terakhir belanja*, maka langkah yang diambil adalah mengubah setiap nilai NaN menjadi nilai rata-rata.

### Ubah Nilai Null dengan Rata-Rata

In [ ]:
rata2 = trainFeatures['terakhir_belanja'].mean()
rata2

Didapatkan nilai **47.23338824821526** yang perlu dibulatkan, untuk mengubah nilai NaN dengan nilai ini.

In [ ]:
trainFeatures['terakhir_belanja'] = trainFeatures['terakhir_belanja'].fillna(rata2)
trainFeatures['terakhir_belanja'] = trainFeatures['terakhir_belanja'].astype(int)
print(trainFeatures['terakhir_belanja'].dtype)

Pembulatan nilai pada kolom *terakhir belanja* dilakukan dengan cara pengubahan tipe data dari float menjadi integer.

In [ ]:
# Cek Ulang Nilai Unique
nilaiTerurut = np.sort(trainFeatures['terakhir_belanja'].unique())
nilaiTerurut

## Kolom Belanja Buah

In [ ]:
trainFeatures.head()

In [ ]:
# Mengambil setiap kuartil pendapatan untuk segmentasi pendapatan
qPendapatan1 = trainFeatures.pendapatan.quantile(0.25)
qPendapatan2 = trainFeatures.pendapatan.quantile(0.5)
qPendapatan3 = trainFeatures.pendapatan.quantile(0.75)

In [ ]:
qPendapatan3

In [ ]:
# Tentukan batas-batas segmen pendapatan
batas_pendapatan = [0, qPendapatan1, qPendapatan2, qPendapatan3, float('inf')]  # 'inf' menunjukkan tak terhingga

# Tentukan label untuk masing-masing segmen
label_segmen = ['rendah', 'sedang', 'tinggi', 'sangat tinggi']

# Buat kolom 'segmen_pendapatan' berdasarkan segmentasi pendapatan
trainFeatures['segmen_pendapatan'] = pd.cut(trainFeatures['pendapatan'], bins=batas_pendapatan, labels=label_segmen)
trainFeatures.head()


In [ ]:
trainFeatures.pendapatan.isnull().sum()

In [ ]:
# # Loop melalui setiap baris data
# for index, row in trainFeatures.iterrows():
#     # Lewati baris data jika nilai 'belanja_buah' sudah terisi atau jika nilai 'pendapatan' adalah NaN
#     if not np.isnan(row['belanja_buah']) or np.isnan(row['pendapatan']):
#         continue
    
#     # Pisahkan fitur (X)
#     X = np.array(row['pendapatan']).reshape(1, -1)
    
#     # Pisahkan data segmen berdasarkan nilai segmen pada baris saat ini
#     segment_df = trainFeatures[trainFeatures['segmen_pendapatan'] == row['segmen_pendapatan']]
    
#     # Filter data yang memiliki nilai non-null dalam kolom 'belanja_buah' di dalam segmen ini
#     valid_data = segment_df.dropna(subset=['belanja_buah'])
#     if valid_data.empty:
#         continue  # Lewati baris data jika tidak ada data yang valid
    
#     # Pisahkan fitur (X) dan target (y) dari data yang valid
#     X_train = np.array(valid_data['pendapatan']).reshape(-1, 1)
#     y_train = np.array(valid_data['belanja_buah'])
    
#     # Train model Random Forest
#     rf_model = RandomForestRegressor()
#     rf_model.fit(X_train, y_train)
    
#     # Prediksi nilai 'belanja_buah'
#     predicted_value = rf_model.predict(X)[0]
    
#     # Isi nilai yang diprediksi ke dalam kolom 'belanja_buah'
#     trainFeatures.at[index, 'belanja_buah'] = predicted_value

In [ ]:
for segmen in trainFeatures['segmen_pendapatan'].unique():
    # Pisahkan data segmen berdasarkan nilai segmen pada baris saat ini
    segment_df = trainFeatures[trainFeatures['segmen_pendapatan'] == segmen]
    
    # Filter data yang memiliki nilai non-null dalam kolom 'belanja_buah' di dalam segmen ini
    valid_data = segment_df.dropna(subset=['belanja_buah'])
    if valid_data.empty:
        continue  # Lewati baris data jika tidak ada data yang valid
    
    # Pisahkan fitur (X) dan target (y) dari data yang valid
    X_train = np.array(valid_data['pendapatan']).reshape(-1, 1)
    y_train = np.array(valid_data['belanja_buah'])
    
    # Train model Random Forest
    rf_model = RandomForestRegressor()
    rf_model.fit(X_train, y_train)
    
    for i, row in trainFeatures.iterrows():
        # Seleksi nilai hanya yang null
        if np.isnan(row['belanja_buah']):
            # Prediksi nilai 'belanja_buah'
            predicted_value = rf_model.predict(np.array(row['pendapatan']).reshape(1, -1))
            trainFeatures.at[i, 'belanja_buah'] = predicted_value

In [ ]:
trainFeatures.belanja_buah.isna().sum()

In [ ]:
# Hitung quartile 1 (Q1) dan quartile 3 (Q3) hanya untuk kolom 'belanja_buah'
Q1 = np.percentile(trainFeatures['belanja_buah'].dropna(), 25)
Q3 = np.percentile(trainFeatures['belanja_buah'].dropna(), 75)

# Hitung IQR
IQR = Q3 - Q1

# Tentukan batas bawah dan batas atas untuk outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identifikasi outliers hanya dalam kolom 'belanja_buah'
outliers = trainFeatures[(trainFeatures['belanja_buah'] < lower_bound) | (trainFeatures['belanja_buah'] > upper_bound)]
print("Banyak outliers:", len(outliers))

In [ ]:
# Plot box plot untuk kolom 'pendapatan'
plt.figure(figsize=(8, 6))  # Atur ukuran plot
plt.boxplot(trainFeatures['belanja_buah'], vert=False)  # vert=False untuk membuat horizontal box plot

# Tambahkan judul plot
plt.title('Box Plot Pendapatan')

# Tambahkan label sumbu y
plt.xlabel('Pendapatan')

# Tampilkan plot
plt.show()

# Exploratory Data Analysis (EDA)

## Analisis Outliers

In [ ]:
while True:
    Q1 = trainFeatures['pendapatan'].quantile(0.25)
    Q3 = trainFeatures['pendapatan'].quantile(0.75)
    IQR = Q3 - Q1

    Upper_Fence = Q3 + 1.5 * IQR
    Lower_Fence = Q1 - 1.5 * IQR

    outliers = trainFeatures[(trainFeatures['pendapatan'] < Lower_Fence) | (trainFeatures['pendapatan'] > Upper_Fence)]['pendapatan']

    if outliers.empty:
        print("Tidak ada outlier lagi.")
        break
    else:
        trainFeatures = trainFeatures[(trainFeatures['pendapatan'] >= Lower_Fence) & (trainFeatures['pendapatan'] <= Upper_Fence)]

In [ ]:
plt.figure(figsize=(10, 6))
plt.boxplot(trainFeatures['pendapatan'])
plt.ylabel('Nilai Pendapatan')
plt.title('Boxplot Pendapatan setelah Penanganan Outlier')
plt.xticks([1], ['Pendapatan'])
plt.show()